In [1]:
%logstop
%logstart -ortq ~/.logs/dw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [60]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [61]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [62]:
import pandas as pd
import numpy as np
import gzip

In [63]:
# load the 2017 data
scripts = pd.read_csv('./dw-data/201701scripts_sample.csv.gz')
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [64]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = pd.read_csv('./dw-data/practices.csv.gz', names=col_names)
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [65]:
chem = pd.read_csv('dw-data/chem.csv.gz')
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [66]:
scripts.select_dtypes(include='number').head()

,items,nic,act_cost,quantity
0,1,0.39,0.47,12
1,1,4.38,4.07,30
2,1,1.50,1.40,1
3,1,2.62,2.44,150
4,1,0.16,0.26,6


In [67]:
cols = ['items', 'quantity', 'nic', 'act_cost']
summary_stats = ['mean', 'std', '25%', '50%', '75%']
df = scripts[cols].describe().loc[summary_stats]
s = scripts[cols].sum().rename('total')

In [68]:
df2 = df.T

In [69]:
#df2['total'] = df2['count'] * df2['mean']
#df2.drop('count', axis=1)

In [70]:
summary = pd.concat([s, df.T], axis=1)
summary_stats = [(t[0], t[1:]) for t in summary.itertuples(name=None)]

In [71]:
#summary_stats = [('items', (0,) * 6), ('quantity', (0,) * 6), ('nic', (0,) * 6), ('act_cost', (0,) * 6)]

In [72]:
grader.score.dw__summary_statistics(summary_stats)

Your score:  1.0


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [73]:
scripts.head(2)

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30


In [74]:
totals = scripts.groupby('bnf_name')['items'].sum()
max_item = totals.idxmax()
most_common_item = [(max_item, totals[max_item])]
print(most_common_item)

[('Omeprazole_Cap E/C 20mg', 218583)]


In [75]:
grader.score.dw__most_common_item(most_common_item)

Your score:  1.0


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [76]:
up = (practices.sort_values('post_code')
  .groupby('code')
  .first()
  .reset_index())

joined = scripts.merge(up[['code','post_code']],
                   left_on = 'practice' , right_on = 'code',
                   how= 'left')

post_item_totals = joined.groupby(['post_code','bnf_name'])['items'].sum().reset_index()
max_idx = post_item_totals.groupby('post_code')['items'].idxmax()
max_items = post_item_totals.loc[max_idx].set_index('post_code')
postal_totals = joined.groupby('post_code')['items'].sum().rename('post_totals')
max_items = pd.concat([max_items, postal_totals],1)
max_items['proportion'] = max_items['items'] / max_items['post_totals']
max_items.drop(['post_totals','items'],1, inplace = True)
items_by_region =list(max_items.head(100).itertuples(name  =  None))

In [77]:
#unique_practices = (practices.sort_values('post_code')
#                    .groupby('code')
#                    .first()
#                    .reset_index())

In [78]:
#joined = scripts.merge(unique_practices[['code', 'post_code']],
#                       left_on='practice', right_on='code',
#                       how='left')

In [79]:
#post_item_totals = joined.groupby(['post_code', 'bnf_name'])['items']\
#                        .sum().reset_index()

In [80]:
#max_idx = (post_item_totals.groupby('post_code')['items']
#           .idxmax().head())

In [81]:
#max_items = post_item_totals.loc[max_idx].set_index('post_code')

In [82]:
#post_totals = joined.groupby('post_code')['items'].sum().rename('post_items')

In [83]:
#max_items = pd.concat([max_items, post_totals], axis=1)

In [84]:
#max_items['proportion'] = max_items['items'] / max_items['post_items']

In [85]:
#max_items.drop(['items', 'post_items'], axis=1, inplace=True)

In [86]:
#items_by_region = list(max_items.itertuples(name=None))[:100]

In [87]:
#items_by_region = [("B11 4BW", "Salbutamol_Inha 100mcg (200 D) CFF", 0.0310589063)] * 100

In [88]:
grader.score.dw__items_by_region(items_by_region)

Your score:  1.0


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [112]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

In [113]:
mask = chem['NAME'].str.contains('|'.join(opioids), case=False)
bad_drugs = chem[mask]['CHEM SUB'].unique()

scripts['opioid'] = scripts['bnf_code'].isin(bad_drugs).astype(int)

In [114]:
#chem['opioids'] = chem['NAME'].str.contains('|'.join(opioids), case=False).astype(int)

#scripts = scripts.merge(chem, left_on='bnf_code',
#                      right_on='CHEM SUB',
#                      how='left')

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [116]:
opioids_per_practice = scripts.groupby('practice')['opioid'].mean()

In [117]:
 relative_opioids_per_practice = opioids_per_practice - scripts['opioid'].mean()

In [118]:
relative_opioids_per_practice.head(2)

practice
A81005   -0.002624
A81007    0.007526
Name: opioid, dtype: float64

Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [121]:
standard_error_per_practice = scripts['opioid'].std() / np.sqrt(scripts.practice.value_counts())
opioid_scores = relative_opioids_per_practice / standard_error_per_practice

In [122]:
opioid_scores.head()

A81005   -0.548306
A81007    1.544557
A81011    2.291795
A81012    1.373060
A81017    0.583168
dtype: float64

In [123]:
anomalies = opioid_scores.rename('score').to_frame().join(unique_practices, how='left')
anomalies = anomalies.join(scripts['practice'].value_counts(), how='left')

In [124]:
scripts['practice'].value_counts()

N83028    2844
L83100    2797
D81043    2459
B81008    2396
B81026    2347
          ... 
Y03010       1
C85617       1
Y03179       1
Y05366       1
Y01174       1
Name: practice, Length: 856, dtype: int64

In [125]:
anomalies = anomalies.sort_values('score', ascending=False).head(100)
anomalies = anomalies[['name', 'score', 'practice']]
anomalies.head(2)

,name,score,practice
Y01852,NATIONAL ENHANCED SERVICE,11.695818,7
Y03006,OUTREACH SERVICE NH / RH,7.339043,2


The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [126]:
#anomalies = [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

In [127]:
anomalies = list(anomalies.itertuples(index=False, name=None))
anomalies[:5]

[('NATIONAL ENHANCED SERVICE', 11.695817862936027, 7),
 ('OUTREACH SERVICE NH / RH', 7.339043019238823, 2),
 ('BRISDOC HEALTHCARE SERVICES OOH', 6.1505817490838295, 60),
 ('H&R P C SPECIAL SCHEME', 5.123032414033079, 36),
 ('HMR BARDOC OOH', 4.958866438487605, 321)]

In [128]:
#results.head()

In [129]:
grader.score.dw__script_anomalies(anomalies)

Your score:  1.0


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [136]:
scripts16 = pd.read_csv('./dw-data/201606scripts_sample.csv.gz')
scripts16.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85638,0301011R0,Salamol_Inha 100mcg (200 D) CFF (Teva),2,2.92,2.73,2
1,N85638,0301011R0,Easyhaler_Salbutamol Sulf 200mcg (200D),1,6.63,6.15,1
2,N85638,0301020I0,Ipratrop Brom_Inh Soln 500mcg/2ml Ud,1,1.77,1.75,12
3,N85638,0301020I0,Ipratrop Brom_Inh Soln 250mcg/1ml Ud,1,4.47,4.15,20
4,N85638,0302000C0,Clenil Modulite_Inha 50mcg (200D),1,3.70,3.44,1


In [145]:
from collections import Counter

c = Counter(scripts16['bnf_name'].to_list())
list(c.items())[:5]

[('Salamol_Inha 100mcg (200 D) CFF (Teva)', 340),
 ('Easyhaler_Salbutamol Sulf 200mcg (200D)', 185),
 ('Ipratrop Brom_Inh Soln 500mcg/2ml Ud', 342),
 ('Ipratrop Brom_Inh Soln 250mcg/1ml Ud', 176),
 ('Clenil Modulite_Inha 50mcg (200D)', 692)]

In [192]:
pct_growth = (scripts['bnf_name'].value_counts()
            / scripts16['bnf_name'].value_counts() 
            - 1)

pct_growth = pd.concat([pct_growth.rename('pct_growth'),
                       scripts16.bnf_name.value_counts().rename('count16')],
                      axis=1, sort=True)

#norm_pct_growth = (pct_growth 
#                   - scripts['bnf_name'].count()
#                   / scripts16['bnf_name'].count() - 1)

#norm_pct_growth = pd.concat([norm_pct_growth.rename('pct_growth'), scripts16.bnf_name.value_counts().rename('count16')], axis=1, sort=True)

mask = pct_growth['count16'] >= 50
filtered_growth = pct_growth[mask].dropna().sort_values('pct_growth', ascending=False)

extreme_growth = pd.concat([filtered_growth.iloc[:50], filtered_growth.iloc[-50:]])

script_growth = list(extreme_growth.itertuples(index=True, name=None))

In [193]:
filtered_growth.head(5)

,pct_growth,count16
Butec_Transdermal Patch 5mcg/hr,3.467742,62.0
Butec_Transdermal Patch 10mcg/hr,3.000000,69.0
Fostair NEXThaler_Inh 200mcg/6mcg (120D),1.430233,86.0
Pneumococcal_Vac 0.5ml Vl (23 Valent),1.269430,193.0
Spiolto Respimat_Inha2.5/2.5mcg(60D)+Dev,1.269231,52.0


In [194]:
#script_growth = [("Butec_Transdermal Patch 5mcg\/hr", 3.4677419355, 62.0)] * 100

In [195]:
grader.score.dw__script_growth(script_growth)

Your score:  1.0


## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [200]:
scripts[['bnf_code', 'bnf_name']].head()

,bnf_code,bnf_name
0,0106020C0,Bisacodyl_Tab E/C 5mg
1,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g
2,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF
3,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml
4,0401020K0,Diazepam_Tab 2mg


In [202]:
p = 1 / scripts['bnf_code'].nunique()
rates = scripts['bnf_code'].value_counts() / scripts['bnf_code'].count()
rare_codes = rates[rates < .1 * p].index
scripts['rare'] = scripts['bnf_code'].isin(rare_codes)

Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [203]:
rare = scripts[scripts['rare']].groupby('practice')['act_cost'].sum()
total = scripts.groupby('practice')['act_cost'].sum()
rare_cost_prop = (rare / total).fillna(0)

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [204]:
proportion = scripts[scripts['rare']]['act_cost'].sum() / scripts['act_cost'].sum()
relative_rare_cost_prop = rare_cost_prop - proportion

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [205]:
standard_errors = relative_rare_cost_prop.std()

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [206]:
rare_scores = relative_rare_cost_prop / standard_errors

In [207]:
rare_scripts = rare_scores.to_frame().merge(unique_practices,
                                           how='left',
                                           left_index=True,
                                           right_on='code')

In [208]:
rare_scripts = rare_scripts[['name', 'act_cost']]

In [209]:
rare_scripts = list(rare_scripts.sort_values('act_cost', ascending=False).itertuples(name=None))[:100]

In [210]:
grader.score.dw__rare_scripts(rare_scripts)

Your score:  1.0


*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*